#### EX 5.0 ConversationBufferMemory

OpenAI에서 지원하는 기본 API는 langchaine없이 사용가능 하지만 메모리를 지원하지 않는다.  
메모리를 설정해 주지않으면 챗봇은 대화를 기억할수 없기 때문에 이어지는 질문을 이해하지 못한다.

Langchaine 에는 5가지 메모리가 있는데 각각의 장단점이 있다.

> 1.  ConversationBufferMemory - 대화 내용 전체를 저장
> 2.  ConversationBufferWindowMemory - 대화 내용 일부를 저장(설정가능)
> 3.  ConversationSummaryMemory - 대화 내용을 자체적으로 요약해서 저장
> 4.  ConversationSummaryBufferMemory -설정한 메시지 갯수 초과시 오래된 메시지들은 요약
> 5.  ConversationKGMemory -대화의 내용을 확인해서 객체의 주요한 특징을 기억한다.

1. ConversationBufferMemory
   - 대화 내용 전체를 저장하는 메모리
   - 대화 내용이 길어지면 메모리가 커지므로 비효율적인 경우가 많다.
     - 사용자와 대화시 처음부터의 지금까지의 대화를 계속 챗봇에게 전달하는데, 대화가 길어질수록 전달하는 양이 점점 늘어나게 된다.

- 예측이나 텍스트 자동완성을 위한 text completion에 유용.
- 채팅 모델과 작업을 할시 ai, human message 모두 필요

tip) 모든 메모리는 save_context, load_memory_variables 라는 항수를 가지고 있음


In [2]:
from langchain.memory import ConversationBufferMemory

# chat model을 위한게 아니라면 return_messages=False로 한다.
memory = ConversationBufferMemory(return_messages=True)

memory.save_context({"input": "Hi!"}, {"output": "How are you?"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi!'), AIMessage(content='How are you?')]}

#### EX 5.1 ConversationBufferWindowMemory

2. ConversationBufferWindowMemory
   - 대화의 특정 부분만을 저장하는 메모리
     - 최근 5개의 대화를 저장하도록 설정하면 6번째 대화시 가장 오래된 대화가 사라짐
   - 단점은 챗봇이 전체 대화가 아닌 최근 대화에만 집중하게 됨


In [1]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(return_messages=True, k=4)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


add_message(1, 1)

In [2]:
add_message(2, 2)
add_message(3, 3)
add_message(4, 4)

In [3]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='1'),
  AIMessage(content='1'),
  HumanMessage(content='2'),
  AIMessage(content='2'),
  HumanMessage(content='3'),
  AIMessage(content='3'),
  HumanMessage(content='4'),
  AIMessage(content='4')]}

#### EX 5.2 ConversationSummaryMemory

3. ConversationSummaryMemory
   - Message를 그대로 저장하는 것이 아니라 coversation의 요약을 자체적으로 해주는 메모리
     - 초기에는 이전보다 더 많은 토큰과 저장공간을 차지함
     - 대화의 내용이 많아질수록 효과가 있음


In [5]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Sungmin, I live in South Korea", "Wow that is so cool!")

In [7]:
add_message("South Korea is so pretty", "I wish I could go!!!")

get_history()

{'history': 'The human introduces themselves as Sungmin from South Korea. The AI thinks it is cool and wishes it could go to South Korea because it is so pretty and expresses its desire to visit.'}

#### EX 5.3 ConversationSummaryBufferMemory

4. ConversationSummaryBufferMemory
   - ConversationSummaryMemory, ConversationBufferMemory의 결합
     - 메모리에 보내온 메시지를 설정한 갯수 만큼 저장
     - 메모리에 저장된 메시지가 설정한 갯수 초과시 오래된 메시지들은 요약해서 정리


In [8]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=40,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Sungmin, I live in South Korea", "Wow that is so cool!")

get_history()

{'history': [HumanMessage(content="Hi I'm Sungmin, I live in South Korea"),
  AIMessage(content='Wow that is so cool!')]}

In [9]:
add_message("South Korea is so pretty", "I wish I could go!!!")
get_history()

{'history': [SystemMessage(content='The human introduces themselves as Sungmin and mentions that they live in South Korea.'),
  AIMessage(content='Wow that is so cool!'),
  HumanMessage(content='South Korea is so pretty'),
  AIMessage(content='I wish I could go!!!')]}

#### EX 5.4 ConversationKGMemory

5. ConversationKnowledgeGraphMemory
   - 대화 중 객체의 Knowledge Graph를 만든다.
   - 즉 대화의 내용을 확인해서 객체의 주요한 특징을 기억한다.

> Knowledge Graph : 연결된 데이터의 네트워크를 통해 사물 간의 관계와 속성을 표현하는 그래프 기반의 구조


In [10]:
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationKGMemory(
    llm=llm,
    return_messages=True,
)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm Sungmin, I live in South Korea", "Wow that is so cool!")

# KG에서 히스토리를 가지고오지 않고 객체를 가지고 오기 때문에 get_history에 내용이 담기지 않는다.
# get_history()

In [11]:
memory.load_memory_variables({"input": "Who is Sungmin"})

{'history': [SystemMessage(content='On Sungmin: Sungmin is a person. Sungmin lives in South Korea.')]}